# Sequence to Sequence (a.k.a. seq2seq)

**학습목표**
* Encoder Decoder 구조를 이해하고 구현할 줄 안다.
* Seq2Seq에 필요한 전처리를 이해한다.
* **데이터 부족**과, **긴 문장**을 겪어본다.

![이런거](https://raw.githubusercontent.com/KerasKorea/KEKOxTutorial/master/media/28_1.png)
---------------------------------
edu.rayleigh@gmail.com
Special Thanks to : 숙번님 ( [봉수골 개발자 이선비](https://www.youtube.com/channel/UCOAyyrvi7tnCAz7RhH98QCQ) )

In [72]:
!wget http://www.manythings.org/anki/kor-eng.zip

--2023-03-31 07:59:29--  http://www.manythings.org/anki/kor-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 224550 (219K) [application/zip]
Saving to: ‘kor-eng.zip.13’

kor-eng.zip.13      100%[===================>] 219.29K  1.01MB/s    in 0.2s    

2023-03-31 07:59:30 (1.01 MB/s) - ‘kor-eng.zip.13’ saved [224550/224550]



In [73]:
import zipfile
kor_eng = zipfile.ZipFile('kor-eng.zip')
kor_eng.extractall()
kor_eng.close()

In [74]:
import pandas as pd
temp = pd.read_table('kor.txt', names=['Eng', 'Kor', 'license'])
temp.shape

(5749, 3)

In [75]:
temp.head()

,Eng,Kor,license
0,Go.,가.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1,Hi.,안녕.,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
2,Run!,뛰어!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
3,Run.,뛰어.,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
4,Who?,누구?,CC-BY 2.0 (France) Attribution: tatoeba.org #2...


In [76]:
eng_sent = temp['Eng'].tolist()
kor_sent = temp['Kor'].tolist()

In [77]:
print(eng_sent[1000])
print(kor_sent[1000])

My cat is black.
내 고양이는 검은색 고양이야.


# 데이터 준비
0. 단어와 구두점 사이 공백 만들기
1. sos 와 eos
1. tokenizing, idx_seq, padding

## 0. 단어와 구두점 사이 공백 만들기


In [78]:
import unicodedata
import re
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')
def eng_preprocessor(sent):
    # 위에서 구현한 함수를 내부적으로 호출
    sent = unicode_to_ascii(sent.lower())

    # 단어와 구두점 사이에 공백을 만듭니다.
    # Ex) "he is a boy." => "he is a boy ."
    sent = re.sub(r"([?.!,'¿])", r" \1 ", sent)

    # (a-z, A-Z, ".", "?", "!", ",") 이들을 제외하고는 전부 공백으로 변환합니다.
    sent = re.sub(r"[^a-zA-Z!.?']+", r" ", sent)

    sent = re.sub(r"\s+", " ", sent)
    return sent

def kor_preprocessor(sent):
    # 위에서 구현한 함수를 내부적으로 호출
    sent = unicode_to_ascii(sent.lower())

    # 단어와 구두점 사이에 공백을 만듭니다.
    # Ex) "he is a boy." => "he is a boy ."
    sent = re.sub(r"([?.!,'¿])", r" \1 ", sent)

    sent = re.sub(r"\s+", " ", sent)
    return sent

In [79]:
eng_preprocessor("I'm just a poor boy.")

"i ' m just a poor boy . "

In [80]:
eng_sent = [ eng_preprocessor(sent) for sent in eng_sent ]
kor_sent = [ kor_preprocessor(sent) for sent in kor_sent ]

In [81]:
print(eng_sent[1000])
print(kor_sent[1000])

my cat is black . 
내 고양이는 검은색 고양이야 . 


## 1. sos 와 eos
1. sos : start of speech
2. eos : end of speech

In [82]:
######################
### Your Code here ###
######################

## 영어 문장 전 후에 <sos>와 <eos>를 추가할 것
## 띄어쓰기 주의!
eng_sent = [f"<sos> {sent} <eos>" for sent in eng_sent]
eng_sent[1000]

'<sos> my cat is black .  <eos>'

## 2. Tokenizing, idx_seq, padding

In [83]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [84]:
######################
### Your Code here ###
######################

# Tokenizing    # 한국어는 lower = False
tokenizer_en = Tokenizer(filters="", lower=True)
tokenizer_en.fit_on_texts(eng_sent)
tokenizer_kr = Tokenizer(filters="", lower=False)
tokenizer_kr.fit_on_texts(kor_sent)

In [85]:
######################
### Your Code here ###
######################

# Index Sequence
eng_seq = tokenizer_en.texts_to_sequences(eng_sent)
kor_seq = tokenizer_kr.texts_to_sequences(kor_sent)

print(eng_seq[1000])
print(kor_seq[1000])

[1, 24, 138, 11, 560, 3, 2]
[12, 257, 1109, 1087, 1]


In [86]:
######################
### Your Code here ###
######################
## 최대 문장 길이에 맞춰지도록 할 것.
# padding
eng_pad = pad_sequences(eng_seq, padding='post')
kor_pad = pad_sequences(kor_seq, padding='pre')

print(eng_pad.shape)
print(kor_pad.shape)

(5749, 104)
(5749, 95)


In [87]:
# tokenizer에서 0 index가 구성되어있지 않지만, 
# pad_sequence에서 pad의 의미로 0을 사용하고 있어서, 전체 사이즈를 구할 때, +1을 해준다.

eng_vocab_size = len(tokenizer_en.word_index) + 1
kor_vocab_size = len(tokenizer_kr.word_index) + 1
print("영어 단어 집합의 크기: {:d}\n한국어 단어 집합의 크기: {:d}".format(eng_vocab_size, kor_vocab_size))

영어 단어 집합의 크기: 3079
한국어 단어 집합의 크기: 7842


# 모델링!

1. 모든 임베딩 레이어는 128개 차원으로 구성.
2. 인코더도 디코더도 GRU, 히든스테이트 512로 구성.
3. 디코더의 GRU 뒤에는 Fully Conneceted layer 사용. 노드 512개
4. 적절한 아웃풋레이어
    * 매 시점, 가장 적절한 단어가 무엇일지 분류 한다고 생각하면 됨!

In [88]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Dense, GRU

In [89]:
######################
### Your Code here ###
######################

# 혹시 이미 그려둔 그래프가 있다면 날려줘!
tf.keras.backend.clear_session()

# 한국어 단어 집합의 크기 : 5551, (50000, 95)
# 영어 단어 집합의 크기 : 2484, (50000, 104)

# 모든 임베딩 레이어는 128개 차원으로 구성.
embedding_dim = 128

# Encoder
enc_X = Input(shape=[kor_pad.shape[1]])
enc_E = Embedding(kor_vocab_size, embedding_dim)(enc_X)
enc_S_full, enc_S = GRU(512, return_sequences=True, return_state=True)(enc_E)

# Decoder
dec_X = Input(shape=[eng_pad.shape[1] - 1])
dec_E = Embedding(eng_vocab_size, embedding_dim)(dec_X)
dec_H = GRU(512, return_sequences=True)(dec_E, initial_state=enc_S)
dec_H = Dense(512, activation='swish')(dec_H)
dec_Y = Dense(eng_vocab_size, activation='softmax')(dec_H)

# Model
model = tf.keras.models.Model([enc_X, dec_X], dec_Y)

model.compile(loss='sparse_categorical_crossentropy',
              optimizer = 'rmsprop',
              metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 95)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 103)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 95, 128)      1003776     ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, 103, 128)     394112      ['input_2[0][0]']                
                                                                                              

In [90]:
######################
### Your Code here ###
######################
## 학습 시킬 것!
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='loss', patience=20, restore_best_weights=True, verbose=1)
model.fit([kor_pad, eng_pad[:, :-1]], eng_pad[:, 1:], epochs=500, batch_size=128, callbacks=[es])

Epoch 1/500
45/45 [==============================] - 13s 186ms/step - loss: 1.3451 - accuracy: 0.9016
Epoch 2/500
45/45 [==============================] - 7s 148ms/step - loss: 0.4902 - accuracy: 0.9227
Epoch 3/500
45/45 [==============================] - 6s 138ms/step - loss: 0.4398 - accuracy: 0.9248
Epoch 4/500
45/45 [==============================] - 6s 132ms/step - loss: 0.4080 - accuracy: 0.9314
Epoch 5/500
45/45 [==============================] - 6s 124ms/step - loss: 0.3963 - accuracy: 0.9369
Epoch 6/500
45/45 [==============================] - 5s 122ms/step - loss: 0.3845 - accuracy: 0.9402
Epoch 7/500
45/45 [==============================] - 6s 126ms/step - loss: 0.3742 - accuracy: 0.9425
Epoch 8/500
45/45 [==============================] - 5s 121ms/step - loss: 0.3683 - accuracy: 0.9433
Epoch 9/500
45/45 [==============================] - 5s 122ms/step - loss: 0.3639 - accuracy: 0.9435
Epoch 10/500
45/45 [==============================] - 5s 122ms/step - loss: 0.3603 - accur

In [91]:
import numpy as np

# 한국어 단어 집합의 크기 : 5551, (50000, 95)
# 영어 단어 집합의 크기 : 2484, (50000, 104)

def translate(kor):
    # eng => index => pad
    kor_seq = tokenizer_kr.texts_to_sequences([kor])
    kor_pad = tf.keras.preprocessing.sequence.pad_sequences(kor_seq, maxlen=95, padding='pre')

    eng = []
    for n in range(104-1):
        # kor => index => pad
        eng_seq = tokenizer_en.texts_to_sequences([['<sos>'] + eng])
        eng_pad = tf.keras.preprocessing.sequence.pad_sequences(eng_seq, maxlen=104-1, padding='post')
        eng_next = model.predict([kor_pad, eng_pad])

        # onehot -> index -> word
        eng = [tokenizer_en.index_word[i] for i in np.argmax(eng_next[0], axis=1) if i != 0]
        # 번역된 word 선택
        eng = eng[:n+1]
        # print(eng)
        if eng[-1] == '<eos>':
            break

    return eng

In [92]:
import random

# 랜덤 10개
indices = list(range(3648))
random.shuffle(indices)

for n in indices[:10]:
    print(f"한국어: {kor_sent[n]}\n영어: {eng_sent[n]}")
    print(f"번역: {' '.join(translate(kor_sent[n])[:-1])}")
    print()

한국어: 난 널 용서한다 . 
영어: <sos> i forgive you .  <eos>
1/1 [==============================] - 0s 25ms/step
번역: i speak french a lot .

한국어: 허튼 소리 . 
영어: <sos> that ' s nonsense .  <eos>
1/1 [==============================] - 0s 24ms/step
번역: it ' s so simple .

한국어: 그는 경찰에 수배되었다 . 
영어: <sos> he ' s wanted by the police .  <eos>
1/1 [==============================] - 0s 24ms/step
번역: he is a famous artist .

한국어: 톰이 숨을 내쉬었어 . 
영어: <sos> tom exhaled .  <eos>
1/1 [==============================] - 0s 25ms/step
번역: tom exhaled .

한국어: 톰은 소심한 어린이야 . 
영어: <sos> tom is a timid child .  <eos>
1/1 [==============================] - 0s 26ms/step
번역: tom is a timid kid .

한국어: 톰은 혼자서 앉아 있었다 . 
영어: <sos> tom sat alone .  <eos>
1/1 [==============================] - 0s 26ms/step
번역: tom sat alone .

한국어: 핸드폰을 가지고 계신가요 ? 
영어: <sos> do you have a cell phone ?  <eos>
1/1 [==============================] - 0s 23ms/step
번역: